In [1]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from tqdm.auto import tqdm

# Clear all previous states and cache
gc.collect()
torch.cuda.empty_cache()


2024-06-18 19:47:27.563993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 19:47:27.564099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 19:47:27.816584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Step 1: PDF Extraction
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text


In [3]:
# Step 2: Load the SQuAD dataset
squad_dataset = load_dataset('squad')

# Select a small subset of the dataset for quicker training
small_train_dataset = squad_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = squad_dataset["validation"].shuffle(seed=42).select(range(200))

# Step 3: Load a pre-trained transformer model
model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Select a small subset of the dataset for quicker training
small_train_dataset = squad_dataset["train"].shuffle(seed=42).select(range(40000))
small_eval_dataset = squad_dataset["validation"].shuffle(seed=42).select(range(8000))

In [5]:
# pre trained transformer model
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_train = small_train_dataset.map(preprocess_function, batched=True, remove_columns=small_train_dataset.column_names)
tokenized_eval = small_eval_dataset.map(preprocess_function, batched=True, remove_columns=small_eval_dataset.column_names)


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [7]:

from torch.utils.data import DataLoader
from transformers import default_data_collator, AdamW
from tqdm.auto import tqdm

In [8]:
train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=16, collate_fn=default_data_collator)
eval_dataloader = DataLoader(tokenized_eval, batch_size=16, collate_fn=default_data_collator)


In [9]:
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
num_training_steps = num_epochs * (len(train_dataloader) + len(eval_dataloader))
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=num_training_steps//3, gamma=0.1)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)


  0%|          | 0/9129 [00:00<?, ?it/s]

In [11]:
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

# Step 8: Evaluate the Model
qa_pipeline = pipeline('question-answering', model="./fine-tuned-model", tokenizer="./fine-tuned-model")



In [12]:
import shutil

# Zip the fine-tuned model directory
shutil.make_archive('fine-tuned-model', 'zip', './fine-tuned-model')

# After running the cell, you can download the zip file from the Kaggle output

    

'/kaggle/working/fine-tuned-model.zip'

In [29]:
context = 'jack is a bouncer at pub'
question = 'who is bouncer at pub ?'

In [30]:
result = qa_pipeline(question=question, context=context)

In [32]:
print(f"Answer: {result['answer']}")

Answer: jack


In [19]:
!pip install pypdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
from pypdf import PdfReader

In [22]:
def extract_text_from_pdf(pdf_path):
  extr = PdfReader(pdf_path)
  text = ''
  for i in range(14,18):
    page = extr.pages[i]
    text_p = page.extract_text()
    text += text_p
  return text